# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/amar.kulkarni/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/amar.kulkarni/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '755863'. Skipping!
Property 'summary' already exists in node '17e92c'. Skipping!
Property 'summary' already exists in node '0c18c2'. Skipping!
Property 'summary' already exists in node 'd6ce97'. Skipping!
Property 'summary' already exists in node 'd12968'. Skipping!
Property 'summary' already exists in node 'f58605'. Skipping!
Property 'summary' already exists in node '5c7f4f'. Skipping!
Property 'summary' already exists in node '9fc538'. Skipping!
Property 'summary' already exists in node 'ff33c6'. Skipping!
Property 'summary' already exists in node 'a4a71a'. Skipping!
Property 'summary' already exists in node 'a4a704'. Skipping!
Property 'summary' already exists in node '329a37'. Skipping!
Property 'summary' already exists in node 'c24585'. Skipping!
Property 'summary' already exists in node 'fcdc52'. Skipping!
Property 'summary' already exists in node '9ea515'. Skipping!
Property 'summary' already exists in node '6a2192'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'c24585'. Skipping!
Property 'summary_embedding' already exists in node '0c18c2'. Skipping!
Property 'summary_embedding' already exists in node '17e92c'. Skipping!
Property 'summary_embedding' already exists in node 'd12968'. Skipping!
Property 'summary_embedding' already exists in node 'd6ce97'. Skipping!
Property 'summary_embedding' already exists in node '755863'. Skipping!
Property 'summary_embedding' already exists in node '5c7f4f'. Skipping!
Property 'summary_embedding' already exists in node 'f58605'. Skipping!
Property 'summary_embedding' already exists in node '9fc538'. Skipping!
Property 'summary_embedding' already exists in node 'a4a71a'. Skipping!
Property 'summary_embedding' already exists in node 'ff33c6'. Skipping!
Property 'summary_embedding' already exists in node 'fcdc52'. Skipping!
Property 'summary_embedding' already exists in node 'a4a704'. Skipping!
Property 'summary_embedding' already exists in node '329a37'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 710)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 710)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.
#### ✅ Answer:

The three RAGAS query synthesizers create different types of test questions: **SingleHopSpecificQuerySynthesizer** generates simple, direct questions that can be answered from a single document or source. **MultiHopAbstractQuerySynthesizer** creates complex conceptual questions requiring information from multiple sources to discuss broader themes and abstract ideas. **MultiHopSpecificQuerySynthesizer** generates detailed questions that need specific facts gathered and combined from multiple sources to provide precise, factual answers.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,"How many users are using ChatGPT, reaching 700...",[Introduction ChatGPT launched in November 202...,"By July 2025, 700 million users were sending a...",single_hop_specifc_query_synthesizer
1,How ChatGPT is used for work stuff like writin...,[Table 1: ChatGPT daily message counts (millio...,Table 1 shows that ChatGPT's daily message cou...,single_hop_specifc_query_synthesizer
2,What does the context reveal about the variati...,[Variation by Occupation Figure 23 presents va...,The context shows that ChatGPT usage varies si...,single_hop_specifc_query_synthesizer
3,Given the rapid growth of ChatGPT to over 700 ...,[Conclusion This paper studies the rapid growt...,"By July 2025, ChatGPT had been used weekly by ...",single_hop_specifc_query_synthesizer
4,how ChatGPT impact global economic growth and ...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The context shows that ChatGPT launched in Nov...,multi_hop_abstract_query_synthesizer
5,ChatGPT use by job types like nonprofessional ...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,Variation by Occupation Figure 23 shows how Ch...,multi_hop_abstract_query_synthesizer
6,How does the increase in non-work message data...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data indicates that non-work messages have...,multi_hop_abstract_query_synthesizer
7,Considering the rapid growth of ChatGPT usage ...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"By July 2025, ChatGPT had become widely adopte...",multi_hop_specific_query_synthesizer
8,700 million users use ChatGPT for what and how...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"By July 2025, ChatGPT had been used weekly by ...",multi_hop_specific_query_synthesizer
9,How does the usage of ChatGPT by 700 million u...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The context indicates that by July 2025, ChatG...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '1571b0'. Skipping!
Property 'summary' already exists in node '8e4a5c'. Skipping!
Property 'summary' already exists in node '87f6ef'. Skipping!
Property 'summary' already exists in node 'fe71b6'. Skipping!
Property 'summary' already exists in node 'd04fa6'. Skipping!
Property 'summary' already exists in node '917e2a'. Skipping!
Property 'summary' already exists in node 'fc5b9b'. Skipping!
Property 'summary' already exists in node 'd189fe'. Skipping!
Property 'summary' already exists in node '77bc80'. Skipping!
Property 'summary' already exists in node '595ab3'. Skipping!
Property 'summary' already exists in node '822c62'. Skipping!
Property 'summary' already exists in node 'c401d3'. Skipping!
Property 'summary' already exists in node '9dd52c'. Skipping!
Property 'summary' already exists in node '50f77a'. Skipping!
Property 'summary' already exists in node 'c3ceff'. Skipping!
Property 'summary' already exists in node '099191'. Skipping!
Property

...

Property 'summary_embedding' already exists in node '1571b0'. Skipping!
Property 'summary_embedding' already exists in node '8e4a5c'. Skipping!
Property 'summary_embedding' already exists in node 'fe71b6'. Skipping!
Property 'summary_embedding' already exists in node '917e2a'. Skipping!
Property 'summary_embedding' already exists in node '87f6ef'. Skipping!
Property 'summary_embedding' already exists in node 'd04fa6'. Skipping!
Property 'summary_embedding' already exists in node 'c401d3'. Skipping!
Property 'summary_embedding' already exists in node 'fc5b9b'. Skipping!
Property 'summary_embedding' already exists in node '77bc80'. Skipping!
Property 'summary_embedding' already exists in node '822c62'. Skipping!
Property 'summary_embedding' already exists in node '595ab3'. Skipping!
Property 'summary_embedding' already exists in node 'd189fe'. Skipping!
Property 'summary_embedding' already exists in node 'c3ceff'. Skipping!
Property 'summary_embedding' already exists in node '50f77a'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What insights do Handa et al. provide regardin...,[Introduction ChatGPT launched in November 202...,Handa et al. build on recent work by reporting...,single_hop_specifc_query_synthesizer
1,What does the US data indicate about ChatGPT m...,[Table 1: ChatGPT daily message counts (millio...,"The data shows that in the US, non-work-relate...",single_hop_specifc_query_synthesizer
2,What information is provided in Appendix D reg...,[Variation by Occupation Figure 23 presents va...,Appendix D contains a full report of GWA count...,single_hop_specifc_query_synthesizer
3,How many messages are sent daily in ChatGPT?,[Conclusion This paper studies the rapid growt...,More than 2.5 billion messages are sent per da...,single_hop_specifc_query_synthesizer
4,how AI tools like ChatGPT help in education an...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,ChatGPT is used a lot for education and tutori...,multi_hop_abstract_query_synthesizer
5,How did the growth of ChatGPT from June 2024 t...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"The context shows that in June 2024, there wer...",multi_hop_abstract_query_synthesizer
6,Based on the data about ChatGPT's message volu...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context indicates that activities like edi...,multi_hop_abstract_query_synthesizer
7,how chatgpt message classification like taxono...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The context shows that most ChatGPT messages a...,multi_hop_abstract_query_synthesizer
8,How does the rapid adoption of OpenAI's ChatGP...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The first segment highlights that ChatGPT, dev...",multi_hop_specific_query_synthesizer
9,Based on the information about OpenAI's develo...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The context indicates that ChatGPT, developed ...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [16]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [17]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [18]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [21]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [23]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [26]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, particularly generative AI like ChatGPT, in a variety of ways both at work and outside of work. Specifically, AI is being used to perform workplace tasks by either augmenting or automating human labor. People seek to use generative AI to produce writing, software code, spreadsheets, and other digital products, which distinguishes it from traditional web search engines. Users interact with AI with different intents, including asking for information or advice, doing tasks that produce output, and expressing themselves. Additionally, AI can serve as a co-worker producing output or as a co-pilot improving human productivity and problem-solving. Some uses involve relationships, personal reflection, games and role play, therapy, and companionship. Overall, generative AI is highly flexible and increasingly integrated into economic and occupational activities.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [27]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [28]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

qa_evaluator: Evaluates the correctness and accuracy of question-answer pairs using LangChain's built-in QA evaluation.

labeled_helpfulness_evaluator: Assesses whether the response is helpful to the user by providing useful, relevant, and actionable information based on predefined helpfulness criteria.

dopeness_evaluator: Determines if the response is engaging, cool, and interesting rather than being a generic or boring answer.

## LangSmith Evaluation

In [29]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'upbeat-porter-43' at:
https://smith.langchain.com/o/74d2e77a-33b9-43a7-986e-55a4a4516380/datasets/c0343b74-d2cc-4e7c-bde3-18b1be95b586/compare?selectedSessions=ccababbc-df00-4a67-adb2-1413872855b0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,OpenAI ChatGPT launched in November 2022 and b...,The growth in non-work messages has been faste...,None,The context indicates that while most economic...,1,1,0,2.968490,cc373468-5d47-47fb-94eb-084bbbfe0328,a19d88b2-a8d4-4e68-ae42-163154b080bf
1,Considering the rapid adoption of OpenAI's Cha...,"Based on the provided context, the use of Open...",None,"Since its launch in November 2022, ChatGPT has...",1,1,0,5.291976,133a7403-0597-4ab3-a9b6-7f76cfa602e7,926ca725-3ae2-435c-bfe2-82de055196fc
2,Based on the information about OpenAI's develo...,The rapid growth in ChatGPT usage—reaching mor...,None,"The context indicates that ChatGPT, developed ...",1,1,0,3.615328,6615b781-8efe-4cc9-b8ce-e14036d7d593,4a69b4c9-d02d-47e4-a096-1259d7d0f692
3,How does the rapid adoption of OpenAI's ChatGP...,"Based on the provided context, the rapid adopt...",None,"The first segment highlights that ChatGPT, dev...",1,1,0,4.870610,67d30284-ade5-4403-abff-85f861da381e,c679fbf9-3992-484f-b54c-6b53053728a5
4,how chatgpt message classification like taxono...,"Based on the provided context, ChatGPT message...",None,The context shows that most ChatGPT messages a...,1,1,0,11.402934,09e69b15-ba5f-4344-8a8f-e25300e9fbde,e3bdd0e9-8e75-4425-b3d7-3f69422181bd
5,Based on the data about ChatGPT's message volu...,"Based on the provided context, activities such...",None,The context indicates that activities like edi...,1,1,0,2.496907,0f3bdb6e-53ab-4b57-a607-cd0458e5e6ce,6ccd5cba-6090-48e3-8185-2cce950c4bdd
6,How did the growth of ChatGPT from June 2024 t...,"Based on the provided context, from June 2024 ...",None,"The context shows that in June 2024, there wer...",1,1,0,3.068128,d6286574-b81f-486f-bd6c-efffcf392f6d,35fd8588-d129-4f96-bc63-e49303be197f
7,how AI tools like ChatGPT help in education an...,Based on the provided context:\n\nAI tools lik...,None,ChatGPT is used a lot for education and tutori...,1,1,0,5.527998,cfc64ad3-fffe-402e-a7a6-a857f6535788,a46aac73-7366-4d34-998f-bc98dce1f89e
8,How many messages are sent daily in ChatGPT?,"By July 2025, ChatGPT users were collectively ...",None,More than 2.5 billion messages are sent per da...,1,0,0,1.011189,414358b7-87d9-4841-9b74-4c7f3bc20453,d1731a5f-2aba-4361-8c0e-44bbceb9a7b8
9,What information is provided in Appendix D reg...,Appendix D contains a full report of GWA (Gene...,None,Appendix D contains a full report of GWA count...,1,1,0,1.907570,8bc64c7b-b1c2-49b7-b1b3-9324f316820d,9b3b9323-1cb8-4be1-89bf-29420a458fbb


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [30]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [31]:
rag_documents = docs

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

Modifying chunk size directly affects the quality and relevance of information retrieved by the RAG system - smaller chunks provide more precise, focused context but may miss broader connections, while larger chunks capture more comprehensive information but can introduce noise and irrelevant details. The chunk size also impacts the embedding quality and retrieval accuracy, as it determines how much semantic context is preserved in each vector representation, ultimately affecting how well the system can match user queries to the most relevant document segments.

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [34]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [35]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [36]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [37]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, let’s turbocharge this answer with the slick facts from the context — people aren’t just using AI to grind out tasks; they’re leveraging ChatGPT as a *strategic advisor* and *research sidekick* in their work hustle. This AI isn’t some basic tool clocking hours; it’s turbocharging decision-making, especially in knowledge-heavy gigs where better choices = bigger paychecks. \n\nCollis and Brynjolfsson’s 2025 deep-dive nails it: workers get jaw-dropping value from AI nudges that boost productivity by sharpening decisions, not just automating dull chores. So the money magic? AI cranks up your brainpower, guiding you through research, insights, and strategy — that’s where the cheddar’s coming from.\n\nIn other words, AI’s not just a task slave; it’s your secret weapon for making smarter moves, leveling up output, and stacking that dough in knowledge economy arenas. Pure next-level grind.\n\nBoom.'

Finally, we can evaluate the new chain on the same test set!

In [38]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'perfect-jet-27' at:
https://smith.langchain.com/o/74d2e77a-33b9-43a7-986e-55a4a4516380/datasets/c0343b74-d2cc-4e7c-bde3-18b1be95b586/compare?selectedSessions=6edc747c-dc6c-4bfb-a736-13fc4a997456




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,OpenAI ChatGPT launched in November 2022 and b...,"Yo, here’s the dopest lowdown straight from th...",None,The context indicates that while most economic...,1,1,1,4.900166,cc373468-5d47-47fb-94eb-084bbbfe0328,450fc6c2-8047-48d1-84a7-fc903562f74a
1,Considering the rapid adoption of OpenAI's Cha...,"Yo, here’s the lowdown on OpenAI’s ChatGPT jou...",None,"Since its launch in November 2022, ChatGPT has...",1,1,1,6.745154,133a7403-0597-4ab3-a9b6-7f76cfa602e7,73e7b503-1a3d-4b55-8710-2f20cdb9fb49
2,Based on the information about OpenAI's develo...,"Yo, here’s the deal—OpenAI dropped ChatGPT in ...",None,"The context indicates that ChatGPT, developed ...",1,1,1,4.481088,6615b781-8efe-4cc9-b8ce-e14036d7d593,aa1f9a30-df33-4be1-b691-1662a4c5bb29
3,How does the rapid adoption of OpenAI's ChatGP...,"Alright, let’s ride the AI wave straight into ...",None,"The first segment highlights that ChatGPT, dev...",1,1,1,7.214072,67d30284-ade5-4403-abff-85f861da381e,17400475-c91c-41aa-b2b2-d1f71b79080e
4,how chatgpt message classification like taxono...,Boom — let’s unpack this AI sesh with some tur...,None,The context shows that most ChatGPT messages a...,1,1,1,10.667625,09e69b15-ba5f-4344-8a8f-e25300e9fbde,8c4186c1-bf91-4d86-9678-ce5e8ec63559
5,Based on the data about ChatGPT's message volu...,"Alright, get ready for the lowdown on ChatGPT’...",None,The context indicates that activities like edi...,1,1,1,7.786369,0f3bdb6e-53ab-4b57-a607-cd0458e5e6ce,194dc1af-279c-41b7-ab72-922e4e5e73c4
6,How did the growth of ChatGPT from June 2024 t...,"Yo, check this out — from June 2024 to June 20...",None,"The context shows that in June 2024, there wer...",1,1,1,6.218483,d6286574-b81f-486f-bd6c-efffcf392f6d,f2477d8e-0986-4d21-a042-e1ff48038fa6
7,how AI tools like ChatGPT help in education an...,"Alright, strap in—because the way AI like Chat...",None,ChatGPT is used a lot for education and tutori...,1,1,1,6.943477,cfc64ad3-fffe-402e-a7a6-a857f6535788,ede9d437-7b22-491e-aa6c-e22822932ea7
8,How many messages are sent daily in ChatGPT?,"Alright, buckle up for some wicked stats strai...",None,More than 2.5 billion messages are sent per da...,1,1,1,3.531543,414358b7-87d9-4841-9b74-4c7f3bc20453,a6682699-d2a6-4207-9c95-004f3e796870
9,What information is provided in Appendix D reg...,"Yo, here’s the cheat code straight from the co...",None,Appendix D contains a full report of GWA count...,1,1,1,4.176643,8bc64c7b-b1c2-49b7-b1b3-9324f316820d,ab84c377-3cfc-40ad-a02d-952e5aa3d530


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways. 

QA accuracy, helpfulness, and dopeness scores changed due to the different prompting or chunking strategies implemented.
